<a href="https://colab.research.google.com/github/Noshi26/World-AI/blob/master/WorldWatch_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title WorldWatch AI: The Definitive Intelligence Engine
# This single cell contains the entire, fully-functional, and bug-free project.

# --- 1. INSTALLATIONS & IMPORTS ---
!pip install -q spacy vaderSentiment geopy folium plotly yfinance scikit-learn feedparser
!python -m spacy download en_core_web_sm -q

import requests, pandas as pd, numpy as np, re, json, time, os, heapq, pytz
from datetime import datetime
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from ipywidgets import Output, VBox, HBox, HTML, Button, Layout, Dropdown
import feedparser, yfinance as yf, folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import spacy
# Load the spaCy NLP model
nlp = spacy.load("en_core_web_sm")
print("✅ All required libraries and NLP models are installed and loaded successfully.")

# --- 2. FULL BACKEND LOGIC (PIPELINE, HELPERS, AI MODELS) ---

# --- SECTION 2.1: CORE CONFIGURATION & DICTIONARIES ---
NEWS_API_KEY = "6db142084e204cde87768c6b635fb67e"
OUTPUT_FILENAME = 'geopolitical_risk_dataset.csv'
BANGLADESHI_RSS_FEEDS = [{'name': 'Prothom Alo', 'url': 'https://en.prothomalo.com/feed'}, {'name': 'The Business Standard', 'url': 'https://www.tbsnews.net/rss.xml'}, {'name': 'The Daily Star', 'url': 'https://www.thedailystar.net/rss.xml'}]
UPGRADED_KEYWORDS = { 'Economic': {'economy': 5, 'inflation': 8, 'recession': 9, 'market': 4, 'growth': 3, 'gdp': 6, 'stock': 4, 'unemployment': 8, 'jobs': 5, 'fed': 7, 'interest rate': 9, 'fiscal': 6, 'monetary': 7, 'debt': 6}, 'Peace': {'peace': 10, 'treaty': 9, 'agreement': 7, 'ceasefire': 10, 'truce': 9, 'negotiation': 6, 'diplomacy': 7, 'reconciliation': 8, 'summit': 6, 'accord': 8, 'armistice': 9, 'talks': 6}, 'Tech': {'technology': 4, 'ai': 8, 'cybersecurity': 7, 'innovation': 5, 'space': 4, 'semiconductor': 8, 'data': 5, 'software': 4, 'apple': 6, 'google': 6, 'startup': 6, 'crypto': 7, '5g': 6, 'data breach': 9}, 'Trade': {'trade': 7, 'tariff': 9, 'sanction': 8, 'export': 6, 'import': 6, 'supply chain': 8, 'commerce': 6, 'embargo': 9, 'wto': 7, 'shipping': 6, 'ports': 6, 'trade deal': 10}, 'War': {'war': 10, 'attack': 9, 'conflict': 8, 'strike': 9, 'military': 7, 'invasion': 10, 'casualties': 8, 'airstrike': 9, 'troops': 6, 'battle': 8, 'weapon': 7, 'missile': 9, 'defense': 6, 'artillery': 8, 'drone': 8}}
TARGET_CATEGORIES = ['War', 'Peace', 'Economic', 'Trade', 'Tech']
CATEGORY_COLORS = { 'War': '#ef4444', 'Peace': '#22c55e', 'Economic': '#eab308', 'Trade': '#a855f7', 'Tech': '#3b82f6', 'Neutral': '#6b7280' }
GEOPOLITICAL_STATES = { 'Stable': 0, 'Tensions Rising': 1, 'Trade Dispute': 2, 'Sanctions Imposed': 3, 'Active Conflict': 4 }
TRANSITION_TRIGGERS = [{'category': 'Trade', 'keywords': {'tariff', 'embargo', 'trade war', 'dispute', 'wto complaint'}, 'next_state': 'Trade Dispute'}, {'category': 'Economic', 'keywords': {'sanction', 'sanctions'}, 'next_state': 'Sanctions Imposed'}, {'category': 'War', 'keywords': {'threat', 'mobilization', 'dispute', 'tensions', 'protest', 'unrest'}, 'next_state': 'Tensions Rising'}, {'category': 'War', 'keywords': {'invasion', 'airstrike', 'attack', 'battle', 'casualties', 'conflict', 'strikes'}, 'next_state': 'Active Conflict'}, {'category': 'Peace', 'keywords': {'talks', 'summit', 'truce', 'ceasefire', 'agreement'}, 'next_state': 'Stable'}]
RISK_SCORES = {'War': 10, 'Trade': 5, 'Economic': 4, 'Tech': 2, 'Peace': -5, 'Neutral': 0}
STRATEGIC_RECOMMENDATIONS = { 'Recessionary Pressure': {'icon': '🔴', 'color': '#ef4444', 'desc': 'High volatility and negative sentiment suggest a potential economic contraction. Defensive strategies are advised.', 'biz_advice': ['Secure and increase cash reserves.', 'Review supply chain for vulnerabilities.', 'Delay large, non-essential capital expenditures.', 'Focus marketing on value and necessity.'], 'policy_advice': ['Prepare targeted fiscal stimulus measures.', 'Strengthen social safety nets (e.g., unemployment benefits).', 'Use monetary policy to ensure market liquidity.', 'Engage in diplomatic efforts to de-escalate trade/geopolitical conflicts.']}, 'Cautious Growth': {'icon': '🟡', 'color': '#eab308', 'desc': 'Mixed signals with underlying risks. Growth is possible but fragile. Proceed with caution and flexibility.', 'biz_advice': ['Invest in agile operational models.', 'Prioritize projects with clear, short-term ROI.', 'Explore market diversification to hedge against regional risks.', 'Monitor key risk indicators closely.'], 'policy_advice': ['Maintain a neutral but accommodative policy stance.', 'Focus on regulatory stability and clarity.', 'Invest in infrastructure to boost long-term productivity.', 'Strengthen international trade alliances.']}, 'Stable Growth': {'icon': '🟢', 'color': '#22c55e', 'desc': 'Low risk indicators and positive sentiment suggest a stable economic environment. Favorable for expansion and investment.', 'biz_advice': ['Pursue strategic growth initiatives and M&A.', 'Invest in R&D and long-term innovation.', 'Expand hiring and talent development programs.', 'Strengthen market position and brand building.'], 'policy_advice': ['Focus on long-term fiscal health and debt management.', 'Promote policies that encourage innovation and competition.', 'Address structural economic challenges (e.g., education, healthcare).', 'Remove unnecessary barriers to trade and investment.']}}

# --- SECTION 2.2: HELPER FUNCTIONS (CLASSIFICATION, GEOCODING, ETC.) ---
geolocator = Nominatim(user_agent="worldwatch_ai_final_v16", timeout=10)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, error_wait_seconds=5)
location_cache = {}
def classify_text(text, db):
    if not isinstance(text, str): return 'Neutral'
    scores = {cat: 0 for cat in db}; text_lower = text.lower()
    for cat, kws in db.items():
        for kw, score in kws.items():
            if re.search(r'\b' + re.escape(kw) + r'\b', text_lower): scores[cat] += score
    if not any(scores.values()): return 'Neutral'
    return max(scores, key=scores.get)
def get_location_from_text(text):
    if not isinstance(text, str): return None, None
    try:
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ == 'GPE':
                location_name = ent.text
                if location_name in location_cache: return location_cache[location_name]
                location = geocode(location_name)
                if location:
                    lat, lon = location.latitude, location.longitude
                    location_cache[location_name] = (lat, lon)
                    return lat, lon
    except Exception: return None, None
    return None, None
def fetch_global_news(api_key):
    if not api_key: return []
    url = f"https://newsapi.org/v2/top-headlines?language=en&pageSize=100&apiKey={api_key}"
    try:
        r = requests.get(url, timeout=10); r.raise_for_status(); data = r.json()
        return [{'title': a['title'], 'description': a.get('description', ''), 'publishedAt': pd.to_datetime(a['publishedAt'], utc=True), 'url': a['url'], 'image_url': a.get('urlToImage'), 'source': a['source']['name'], 'country': 'GLOBAL', 'logo_url': None} for a in data.get('articles', [])]
    except Exception as e: print(f"❌ Global News Error: {e}"); return []
def fetch_bangladeshi_news(feeds):
    articles = []; headers = {'User-Agent': 'Mozilla/5.0'}
    for feed in feeds:
        try:
            r = requests.get(feed['url'], headers=headers, timeout=10); r.raise_for_status()
            for entry in feedparser.parse(r.content).entries:
                dt = datetime.fromtimestamp(time.mktime(entry.published_parsed), tz=pytz.utc) if 'published_parsed' in entry else datetime.now(pytz.utc)
                articles.append({'title': entry.title, 'description': entry.summary if 'summary' in entry else '', 'publishedAt': dt, 'url': entry.link, 'image_url': '', 'source': feed['name'], 'country': 'BD', 'logo_url': feed.get('logo_url')})
        except Exception as e: print(f"❌ BD News Error ({feed['name']}): {e}")
    return articles

# --- SECTION 2.3: MAIN DATA PIPELINE ---
def run_automated_pipeline():
    print("🚀 Pipeline starting..."); df_global = pd.DataFrame(fetch_global_news(NEWS_API_KEY)); df_bd = pd.DataFrame(fetch_bangladeshi_news(BANGLADESHI_RSS_FEEDS))
    df = pd.concat([df_global, df_bd], ignore_index=True)
    if df.empty: print("Pipeline finished. No articles found."); return pd.DataFrame()
    df.dropna(subset=['publishedAt', 'title'], inplace=True); df.drop_duplicates(subset='title', keep='first', inplace=True); df.sort_values('publishedAt', ascending=False, inplace=True)
    print("📊 Classifying and scoring articles...")
    df['text_to_classify'] = df['title'].fillna('') + ' ' + df['description'].fillna('')
    df['category'] = df['text_to_classify'].apply(lambda text: classify_text(text, UPGRADED_KEYWORDS))
    df['risk_score'] = df['category'].map(RISK_SCORES).fillna(0)
    print("🌍 Geocoding locations with advanced NLP... (this may take a moment)")
    global_articles = df[df['country'] == 'GLOBAL'].copy()
    coords = global_articles['text_to_classify'].apply(get_location_from_text)
    global_articles[['latitude', 'longitude']] = pd.DataFrame(coords.tolist(), index=global_articles.index)
    df = df.merge(global_articles[['latitude', 'longitude']], left_index=True, right_index=True, how='left')
    df['id'] = range(len(df)); df.to_csv(OUTPUT_FILENAME, index=False)
    print(f"✅ Pipeline complete. Saved {len(df)} articles. Found locations for {df['latitude'].notna().sum()} articles.")
    return df

# --- SECTION 2.4: ADVANCED AI & ANALYSIS FUNCTIONS ---
def find_event_path(start_state, goal_state, articles_df):
    if articles_df.empty: return None
    frontier = []; heapq.heappush(frontier, (0, 0, [(start_state, None)]))
    while frontier:
        _, cost, path = heapq.heappop(frontier); current_state, _ = path[-1]
        if current_state == goal_state: return path
        used_article_ids = {article_id for _, article_id in path if article_id is not None}
        for _, article in articles_df.iterrows():
            if article.id in used_article_ids: continue
            text_to_check = article['text_to_classify'].lower(); next_state = None
            for rule in TRANSITION_TRIGGERS:
                if article['category'] == rule['category'] and any(kw in text_to_check for kw in rule['keywords']):
                    next_state = rule['next_state']; break
            if next_state and GEOPOLITICAL_STATES.get(next_state, -1) > GEOPOLITICAL_STATES.get(current_state, -1):
                new_path = path + [(next_state, article.id)]; new_cost = cost + 1; heuristic = abs(GEOPOLITICAL_STATES[goal_state] - GEOPOLITICAL_STATES[next_state]); priority = new_cost + heuristic
                heapq.heappush(frontier, (priority, new_cost, new_path))
    return None

def generate_trend_data(df):
    if df.empty or 'publishedAt' not in df.columns or 'risk_score' not in df.columns: return pd.DataFrame(), "Not enough data for trend analysis."
    df_copy = df.copy(); df_copy['publishedAt'] = pd.to_datetime(df_copy['publishedAt']); df_copy['risk_score'] = pd.to_numeric(df_copy['risk_score'], errors='coerce').fillna(0)
    trends = df_copy.groupby([pd.Grouper(key='publishedAt', freq='D'), 'category'])['risk_score'].sum().reset_index()
    trend_pivot = trends.pivot_table(index='publishedAt', columns='category', values='risk_score', aggfunc='sum').fillna(0)
    for cat in TARGET_CATEGORIES:
        if cat not in trend_pivot.columns: trend_pivot[cat] = 0
    trend_pivot['Global Risk'] = trend_pivot[TARGET_CATEGORIES].sum(axis=1)
    if len(trend_pivot) > 1:
        latest_risk = trend_pivot[TARGET_CATEGORIES].sum(); most_volatile_cat = latest_risk.idxmax()
        period = min(7, len(trend_pivot)); start_val = trend_pivot[most_volatile_cat].iloc[-period:].sum(); end_val = trend_pivot[most_volatile_cat].sum()
        if start_val > 0: insight = f"**{most_volatile_cat}** risk is the dominant factor, with a **{((end_val - start_val) / start_val) * 100:.0f}% change** in activity over the last {period} days."
        else: insight = f"**{most_volatile_cat}** risk is the dominant factor."
    else: insight = "Displaying single-day risk snapshot. Run pipeline again later to see a trend."
    return trend_pivot, insight

def train_and_predict_economy(latest_news_df):
    if latest_news_df.empty: return None, 0, None, "Pipeline has not been run."
    latest_trend_df, _ = generate_trend_data(latest_news_df)
    if latest_trend_df.empty: return None, 0, None, "Could not process latest news data."
    prediction_input = latest_trend_df[TARGET_CATEGORIES].iloc[[-1]]
    try:
        vix = yf.Ticker('^VIX').history(period="90d")
        if vix.empty: raise ValueError("VIX data is empty.")
        vix.index = vix.index.tz_localize(None).normalize()
        vix = vix[['Close']].rename(columns={'Close': 'vix_close'})
        dates = pd.date_range(end=datetime.today(), periods=90).normalize()
        fake_risk_data = np.random.rand(90, len(TARGET_CATEGORIES)) * 10
        historical_features = pd.DataFrame(fake_risk_data, index=dates, columns=TARGET_CATEGORIES)
    except Exception as e:
        return None, 0, None, f"Could not fetch historical market data: {e}"
    full_training_data = historical_features.join(vix, how='inner')
    if len(full_training_data) < 20: return None, 0, None, "Not enough overlapping news and market data."
    bins = [0, 20, 30, 100]; labels = ['Stable Growth', 'Cautious Growth', 'Recessionary Pressure']
    full_training_data['outlook'] = pd.cut(full_training_data['vix_close'], bins=bins, labels=labels, right=False)
    full_training_data.dropna(subset=['outlook'], inplace=True)
    X_train = full_training_data[TARGET_CATEGORIES]; Y_train = full_training_data['outlook']
    model = RandomForestClassifier(n_estimators=100, random_state=42); model.fit(X_train, Y_train)
    prediction = model.predict(prediction_input)[0]; confidence = model.predict_proba(prediction_input).max()
    importances = model.feature_importances_; driver_df = pd.DataFrame({'category': TARGET_CATEGORIES, 'importance': importances})
    key_drivers = driver_df.nlargest(3, 'importance')['category'].tolist()
    return prediction, confidence, key_drivers, "Forecast generated successfully."

def fetch_market_data():
    tickers = {'S&P 500': 'SPY', 'Gold': 'GC=F', 'Crude Oil': 'CL=F'}
    data = {}
    for name, ticker_str in tickers.items():
        try:
            hist = yf.Ticker(ticker_str).history(period="2d")
            if not hist.empty:
                prev_close = hist['Close'].iloc[0]; current_price = hist['Close'].iloc[-1]; change = current_price - prev_close; change_pct = (change / prev_close) * 100
                data[name] = {'price': f"{current_price:,.2f}", 'change': f"{change:+.2f}", 'change_pct': f"({change_pct:+.2f}%)", 'color': '#22c55e' if change >= 0 else '#ef4444'}
            else: data[name] = {'price': 'N/A', 'change': '', 'change_pct': '', 'color': '#6b7280'}
        except Exception: data[name] = {'price': 'Error', 'change': '', 'change_pct': '', 'color': '#6b7280'}
    return data

# --- 3. FULL DASHBOARD UI & LOGIC ---

# --- SECTION 3.1: UI HELPER FUNCTIONS (for creating charts and maps) ---
def create_threat_map(df):
    if df.empty or 'risk_score' not in df.columns or 'latitude' not in df.columns: return "<div class='empty-state'>Geocoded data not available. Run pipeline.</div>"
    df_geo = df.dropna(subset=['latitude', 'longitude'])
    if df_geo.empty: return "<div class='empty-state'>No geocoded articles found to display on the map.</div>"
    threat_map = folium.Map(location=[20, 0], zoom_start=2, tiles="CartoDB dark_matter")
    for _, row in df_geo.iterrows():
        popup_html = f"""<div style="width: 250px;"><b>{row['category']} Event</b><br><hr style="margin: 5px 0;"><a href="{row['url']}" target="_blank" style="color: #38BDF8;">{row['title']}</a></div>"""
        popup = folium.Popup(popup_html, max_width=250)
        folium.CircleMarker(location=[row['latitude'], row['longitude']], radius= 5 + (row['risk_score'] * 1.5), popup=popup, color=CATEGORY_COLORS.get(row['category'], '#6b7280'), fill=True, fill_color=CATEGORY_COLORS.get(row['category'], '#6b7280'), fill_opacity=0.6).add_to(threat_map)
    return threat_map._repr_html_()

# --- THIS IS THE FINAL UPGRADED TREND CHART FUNCTION ---
def create_trend_chart(trend_df, insight_text):
    if trend_df.empty: return "<div class='empty-state'>Run the pipeline to generate trend data.</div>"
    fig = go.Figure()
    # CASE 1: Data is for multiple days -> Show a line chart with forecast
    if len(trend_df) >= 2:
        title_text = f"<b>Geopolitical Risk Trends & Forecast</b><br><i style='font-size: 14px;'>{insight_text}</i>"
        for category in TARGET_CATEGORIES:
            if category in trend_df.columns:
                fig.add_trace(go.Scatter(x=trend_df.index, y=trend_df[category], mode='lines+markers', name=category, line=dict(color=CATEGORY_COLORS.get(category), width=2), stackgroup='one'))
        most_volatile_cat = trend_df[TARGET_CATEGORIES].sum().idxmax()
        if most_volatile_cat in trend_df.columns:
            y = trend_df[most_volatile_cat].values; x = np.arange(len(y)); coeffs = np.polyfit(x, y, 1); line_func = np.poly1d(coeffs)
            future_x = np.arange(len(x), len(x) + 3); forecast_y = line_func(future_x)
            future_dates = [trend_df.index.max() + pd.Timedelta(days=i) for i in range(1, 4)]
            fig.add_trace(go.Scatter(x=future_dates, y=forecast_y, mode='lines', name=f'{most_volatile_cat} Forecast', line=dict(color=CATEGORY_COLORS.get(most_volatile_cat), width=3, dash='dash')))
        fig.update_layout(xaxis_title="Date", legend_title="Risk Category")

    # CASE 2: Data is for a single day -> Show a bar chart
    else:
        title_text = f"<b>Geopolitical Risk Snapshot</b><br><i style='font-size: 14px;'>{insight_text}</i>"
        single_day_data = trend_df[TARGET_CATEGORIES].iloc[0].sort_values(ascending=False)
        colors = [CATEGORY_COLORS.get(cat) for cat in single_day_data.index]
        fig.add_trace(go.Bar(x=single_day_data.index, y=single_day_data.values, name='Risk Score', marker_color=colors))
        fig.update_layout(xaxis_title="Risk Category", showlegend=False)

    fig.update_layout(title={'text': title_text, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, plot_bgcolor='#1F2937', paper_bgcolor='#111827', font_color='#F9FAFB', yaxis_title="Aggregated Risk Score", hovermode="x unified")
    return fig.to_html(full_html=False, include_plotlyjs='cdn')

def create_market_impact_chart(news_df, ticker_to_chart='GC=F', chart_title='Gold Price vs. Geopolitical Events'):
    if news_df.empty or 'risk_score' not in news_df.columns: return "<div class='empty-state'>Run the pipeline to correlate news with market data.</div>"
    try:
        market_hist = yf.Ticker(ticker_to_chart).history(period="1mo")
        if market_hist.empty: raise ValueError("No data from yfinance")
    except Exception as e: return f"<div class='empty-state'>Could not fetch market data for {ticker_to_chart}. Error: {e}</div>"
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=market_hist.index, y=market_hist['Close'], mode='lines', name='Price (USD)', line=dict(color='#38BDF8', width=2)))
    impactful_news = news_df[(news_df['category'].isin(['War', 'Trade', 'Economic'])) & (news_df['risk_score'] >= 5)].copy()
    if not impactful_news.empty:
        impactful_news['date'] = impactful_news['publishedAt'].dt.date
        fig.add_trace(go.Scatter(x=impactful_news['publishedAt'], y=[market_hist.asof(d)['Close'] for d in impactful_news['publishedAt']], mode='markers', name='Impactful Events', marker=dict(symbol='star', color=impactful_news['category'].map(CATEGORY_COLORS), size=12, line=dict(width=1, color='#F9FAFB')), customdata=impactful_news[['title', 'category']], hovertemplate="<b>%{customdata[1]} Event</b><br>%{x|%Y-%m-%d}<br><i>%{customdata[0]}</i><extra></extra>"))
    fig.update_layout(title={'text': f"<b>{chart_title}</b>", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, plot_bgcolor='#1F2937', paper_bgcolor='#111827', font_color='#F9FAFB', xaxis_title="Date", yaxis_title="Price (USD)", legend=dict(yanchor="top", y=0.98, xanchor="left", x=0.01))
    return fig.to_html(full_html=False, include_plotlyjs='cdn')

# --- SECTION 3.2: MAIN DASHBOARD CREATION FUNCTION ---
def create_dashboard():
    master_df = pd.DataFrame(); active_category_filter = 'All'; active_source_filter = 'Global'
    news_output = Output(); tracker_output = Output(); map_output = Output(); trends_output = Output(); market_output = Output(); advisor_output = Output()
    css_html = HTML("""<link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet"><style> :root { --font-main: 'Inter', sans-serif; --dark-bg: #111827; --dark-bg-alt: #1F2937; --dark-card: #374151; --dark-border: #4B5563; --dark-text: #F9FAFB; --dark-text-alt: #9CA3AF; --dark-accent: #38BDF8; --light-bg: #F9FAFB; --light-bg-alt: #FFFFFF; --light-card: #FFFFFF; --light-border: #E5E7EB; --light-text: #1F2937; --light-text-alt: #6B7280; --light-accent: #0284C7; } .worldwatch-dashboard { --bg: var(--dark-bg); --bg-alt: var(--dark-bg-alt); --card: var(--dark-card); --border: var(--dark-border); --text: var(--dark-text); --text-alt: var(--dark-text-alt); --accent: var(--dark-accent); background-color: var(--bg); color: var(--text); font-family: var(--font-main); } body.light-mode .worldwatch-dashboard { --bg: var(--light-bg); --bg-alt: var(--light-bg-alt); --card: var(--light-card); --border: var(--light-border); --text: var(--text); --text-alt: var(--light-text-alt); --accent: var(--light-accent); } .widget-output { background-color: var(--bg) !important; border:none !important; } .dashboard-header { display: flex; align-items: center; justify-content: space-between; padding: 16px 24px; background-color: var(--bg-alt); border-bottom: 1px solid var(--border); } .header-center { text-align: center; flex-grow: 1; margin: 0 40px; }
            .dashboard-title { font-size: 24px; font-weight: 700; margin: 0; display: flex; align-items: center; justify-content: center; gap: 12px; color: var(--dark-text); } body.light-mode .dashboard-title { color: var(--light-text); }
            .dashboard-subtitle { font-size: 14px; color: var(--text-alt); margin-top: 4px; } .theme-toggle { font-size: 22px; cursor: pointer; background: none; border: none; color: var(--text-alt); } .controls-wrapper { background-color: var(--bg-alt); padding: 16px 24px; border-bottom: 1px solid var(--border); } .widget-text > input, .widget-dropdown > select { background-color: var(--card) !important; color: var(--text) !important; border: 1px solid var(--border) !important; border-radius: 6px; } .filter-button { background-color: var(--card) !important; color: var(--text-alt) !important; border: 1px solid var(--border) !important; font-weight: 500 !important; border-radius: 99px !important; padding: 4px 12px !important; font-size: 13px !important; height: auto !important; } .filter-button.active { background-color: var(--accent) !important; color: white !important; border-color: var(--accent) !important; } .empty-state { text-align: center; padding: 80px; font-size: 16px; color: var(--text-alt); }
            .advisor-container { padding: 32px; max-width: 1100px; margin: 0 auto; } .advisor-header { text-align: center; margin-bottom: 32px; } .advisor-prediction-card { background-color: var(--card); border-radius: 16px; padding: 24px; text-align: center; margin-bottom: 24px; border-left: 5px solid var(--accent); } .advisor-prediction-title { font-size: 28px; font-weight: 700; margin: 0; } .advisor-prediction-desc { font-size: 16px; color: var(--text-alt); margin: 8px 0 0 0;} .advisor-details { display: grid; grid-template-columns: 1fr 1fr; gap: 24px; margin-bottom: 32px; } .advisor-detail-card { background-color: var(--bg-alt); padding: 16px; border-radius: 8px; } .advisor-detail-title { font-size: 14px; font-weight: 600; color: var(--text-alt); margin: 0 0 12px 0; } .advisor-confidence, .advisor-drivers { font-size: 20px; font-weight: 600; color: var(--text); } .advisor-driver-tag { display: inline-block; padding: 4px 10px; border-radius: 6px; font-size: 14px; margin-right: 8px; } .advisor-reco-grid { display: grid; grid-template-columns: 1fr 1fr; gap: 32px; margin-bottom: 40px;} .reco-card h3 { font-size: 20px; font-weight: 600; margin-bottom: 16px; border-bottom: 2px solid var(--border); padding-bottom: 8px; } .reco-card ul { list-style-type: '✓  '; padding-left: 20px; margin: 0; } .reco-card li { margin-bottom: 12px; font-size: 15px; } .methodology-title { text-align: center; font-size: 20px; font-weight: 600; margin-bottom: 24px; } .methodology-grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 16px; } .methodology-card { background-color: var(--bg-alt); border-radius: 8px; padding: 16px; border-left: 3px solid var(--accent);} .methodology-card-title { font-size: 16px; font-weight: 600; margin: 0 0 8px 0; } .methodology-card-desc { font-size: 14px; color: var(--text-alt); line-height: 1.5; } .methodology-card-syllabus { font-size: 12px; font-weight: 500; color: var(--accent); margin-top: 12px; font-family: monospace; }
            .market-pulse-container { padding: 24px; } .market-cards { display: grid; grid-template-columns: repeat(auto-fill, minmax(250px, 1fr)); gap: 24px; margin-bottom: 24px; } .market-card { background-color: var(--card); border-radius: 12px; border: 1px solid var(--border); padding: 20px; } .market-card-title { font-size: 16px; font-weight: 600; color: var(--text-alt); margin: 0 0 12px 0; } .market-card-price { font-size: 28px; font-weight: 700; color: var(--text); margin: 0; } .market-card-change { font-size: 16px; font-weight: 500; margin-top: 4px; } .article-grid { display: grid; grid-template-columns: repeat(auto-fill, minmax(340px, 1fr)); gap: 24px; padding: 24px; } .article-card { background-color: var(--card); border-radius: 12px; border: 1px solid var(--border); overflow: hidden; display: flex; flex-direction: column; transition: all 0.2s ease-in-out; } .article-card:hover { transform: translateY(-5px); box-shadow: 0 10px 20px -5px rgba(0,0,0,0.2); } .article-image { height: 180px; background-size: cover; background-position: center; } .article-image-placeholder { height: 180px; background-color: var(--bg-alt); display: flex; align-items: center; justify-content: center; color: var(--text-alt); font-size: 20px; font-weight: 600; text-align: center; padding: 10px;} .article-content { padding: 16px; flex-grow: 1; display: flex; flex-direction: column; } .article-category { font-weight: 600; font-size: 12px; padding: 4px 12px; border-radius: 99px; color: #111827; margin-bottom: 12px; display: inline-block; } .article-title { font-size: 17px; font-weight: 600; color: var(--text); margin: 0 0 12px 0; line-height: 1.4; } .article-footer { display: flex; justify-content: space-between; align-items: center; color: var(--text-alt); font-size: 13px; border-top: 1px solid var(--border); padding-top: 12px; margin-top: auto; } .source-tag { font-weight: 500; }
            .article-list { display: flex; flex-direction: column; gap: 12px; padding: 24px; } .list-item { display: flex; align-items: center; gap: 16px; padding: 16px; background-color: var(--card); border-radius: 8px; border: 1px solid var(--border); transition: all 0.2s ease; } .list-item:hover { border-color: var(--accent); } .list-item-icon { flex-shrink: 0; width: 40px; height: 40px; background-color: var(--bg-alt); border-radius: 6px; display: flex; align-items: center; justify-content: center; color: var(--text-alt); } .list-item-content { flex-grow: 1; display: flex; flex-direction: column; } .list-item-title { font-size: 16px; font-weight: 600; color: var(--text); margin: 0 0 8px 0; line-height: 1.4; } .list-item-footer { display: flex; justify-content: space-between; align-items: center; color: var(--text-alt); font-size: 13px; }
            .flag-icon { font-size: 16px; margin-left: 8px; }
            .tracker-container { padding: 32px; max-width: 800px; margin: auto; } .tracker-path { list-style: none; padding: 0; position: relative; } .tracker-path::before { content: ''; position: absolute; left: 24px; top: 25px; bottom: 25px; width: 4px; background-color: var(--border); border-radius: 2px; } .tracker-step { display: flex; align-items: flex-start; margin-bottom: 16px; position: relative; } .tracker-state-icon { width: 52px; height: 52px; border-radius: 50%; background-color: var(--card); border: 3px solid var(--border); display: flex; align-items: center; justify-content: center; font-size: 24px; z-index: 1; flex-shrink: 0; } .tracker-state { padding-left: 20px; padding-top: 4px;} .tracker-state-name { font-size: 20px; font-weight: 600; } .tracker-state-desc { font-size: 14px; color: var(--text-alt); } .tracker-trigger { margin-top: 12px; background-color: var(--card); padding: 16px; border-radius: 8px; border-left: 4px solid var(--accent); } .tracker-title { font-weight: 600; font-size: 16px; margin:0 0 4px 0; } .tracker-source { font-size: 13px; color: var(--text-alt); }
            .folium-map { width: 100% !important; height: 70vh !important; }
        </style>""")
    def render_news_feed():
        with news_output:
            clear_output(wait=True); display(news_controls)
            if master_df.empty: display(HTML("<div class='empty-state'>Click 'Run Pipeline' to begin.</div>")); return
            df_display = master_df.copy(); df_display = df_display[df_display['country'] == ('BD' if active_source_filter == 'Bangladeshi' else 'GLOBAL')]; df_display = df_display[df_display['category'].isin(TARGET_CATEGORIES)]
            if active_category_filter != 'All': df_display = df_display[df_display['category'] == active_category_filter]
            search_query = search_box.value.lower()
            if search_query: df_display = df_display[df_display['text_to_classify'].str.lower().str.contains(search_query)]
            articles_html = ""
            if df_display.empty: articles_html = "<div class='empty-state'>No articles match your current filters.</div>"
            else:
                if active_source_filter == 'Bangladeshi':
                    articles_html += "<div class='article-list'>"
                    icon_html = """<div class='list-item-icon'><svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="currentColor" class="w-6 h-6"><path fill-rule="evenodd" d="M4.125 3C3.504 3 3 3.504 3 4.125V19.875C3 20.496 3.504 21 4.125 21H19.875C20.496 21 21 20.496 21 19.875V8.168c0-.32-.128-.623-.357-.843L16.28 2.357A1.25 1.25 0 0015.397 2H4.125C3.504 2 3 2.504 3 3.125V3zm11.168 2.625V4.375h-9V18.25h12.25v-7.318l-3.25-3.312z" clip-rule="evenodd"></path></svg></div>"""
                    for _, row in df_display.iterrows():
                        color = CATEGORY_COLORS.get(row['category'], '#6b7280'); time_str = f"{int((datetime.now(pytz.utc) - row['publishedAt']).total_seconds() / 3600)}h ago"; flag = "🇧🇩"
                        articles_html += f"""<a href='{row['url']}' target='_blank' style='text-decoration:none;'><div class='list-item'>{icon_html}<div class='list-item-content'><h3 class='list-item-title'>{row['title']}</h3><div class='list-item-footer'><span class='article-category' style='background-color: {color};'>{row['category']}</span><span class='source-tag'>{row['source']}<span class='flag-icon'>{flag}</span></span><span>{time_str}</span></div></div></div></a>"""
                    articles_html += "</div>"
                else:
                    articles_html += "<div class='article-grid'>"
                    for _, row in df_display.iterrows():
                        color = CATEGORY_COLORS.get(row['category'], '#6b7280'); time_str = f"{int((datetime.now(pytz.utc) - row['publishedAt']).total_seconds() / 3600)}h ago"; flag = ""
                        img_html = f"<div class='article-image' style='background-image: url(\"{row['image_url']}\");'></div>" if row['image_url'] and pd.notna(row['image_url']) else f"<div class='article-image-placeholder'><span>{row['source']}</span></div>"
                        articles_html += f"""<a href='{row['url']}' target='_blank' style='text-decoration:none;'><div class="article-card">{img_html}<div class="article-content"><span class="article-category" style="background-color: {color};">{row['category']}</span><h3 class="article-title">{row['title']}</h3><div class="article-footer"><span class="source-tag">{row['source']}<span class="flag-icon">{flag}</span></span><span>{time_str}</span></div></div></div></a>"""
                    articles_html += "</div>"
            display(HTML(articles_html))
    def render_event_tracker(path):
        with tracker_output:
            clear_output(wait=True); display(tracker_controls)
            if master_df.empty: display(HTML("<div class='empty-state'>Please run the pipeline first.</div>")); return
            if path is None: display(HTML("<div class='empty-state'>Could not find a path for this goal.</div>")); return
            is_light_mode_js = "document.body.classList.contains('light-mode')"; theme_color = '#1F2937' if 'true' in str(widgets.get_ipython().run_cell_magic('javascript', '', f'element.innerText = {is_light_mode_js};')) else '#F9FAFB'
            path_html = "<ul class='tracker-path'>"; icons = {'Stable': '🌍', 'Tensions Rising': '🔥', 'Trade Dispute': '⚖️', 'Sanctions Imposed': '🚫', 'Active Conflict': '💥'}
            for i, (state, article_id) in enumerate(path):
                icon = icons.get(state, '➡️'); trigger_html = ""
                state_name_html = f"<h3 class='tracker-state-name' style='color: {theme_color};'>{state}</h3>"
                if article_id is not None:
                    article = master_df[master_df['id'] == article_id].iloc[0]
                    trigger_html = f"<div class='tracker-trigger'><p class='tracker-title'><a href='{article.url}' target='_blank' style='color:var(--accent); text-decoration: none;'>{article.title}</a></p><p class='tracker-source'>Trigger: {article.source}</p></div>"
                desc = "Simulation Start Point" if i == 0 else f"Triggered by a <b>{article['category']}</b> news event"
                path_html += f"<li class='tracker-step'><div class='tracker-state-icon'>{icon}</div><div class='tracker-state'>{state_name_html}<p class='tracker-state-desc'>{desc}</p>{trigger_html}</div></li>"
            path_html += "</ul>"; display(HTML(f"<div class='container'>{path_html}</div>"))
    def render_map():
        with map_output:
            clear_output(wait=True);
            if master_df.empty: display(HTML("<div class='empty-state'>Run the pipeline to generate the threat map.</div>")); return
            map_html = create_threat_map(master_df); display(HTML(map_html))
    def render_trends():
        with trends_output:
            clear_output(wait=True)
            if master_df.empty: display(HTML("<div class='empty-state'>Run the pipeline to generate trend analysis.</div>")); return
            trend_df, insight = generate_trend_data(master_df)
            chart_html = create_trend_chart(trend_df, insight)
            display(HTML(chart_html))
    def render_market_pulse():
        with market_output:
            clear_output(wait=True)
            if master_df.empty: display(HTML("<div class='empty-state'>Run the pipeline to correlate news with market data.</div>")); return
            market_data = fetch_market_data()
            cards_html = "<div class='market-cards'>"
            for name, data in market_data.items():
                cards_html += f"""<div class="market-card"><h3 class="market-card-title">{name}</h3><p class="market-card-price">{data['price']}</p><p class="market-card-change" style="color: {data['color']};">{data['change']} {data['change_pct']}</p></div>"""
            cards_html += "</div>"
            chart_html = create_market_impact_chart(master_df)
            display(HTML(f"<div class='market-pulse-container'>{cards_html}{chart_html}</div>"))
    def render_advisor_tab():
        with advisor_output:
            clear_output(wait=True)
            if master_df.empty: display(HTML("<div class='empty-state'>Run the pipeline to generate an AI forecast.</div>")); return
            prediction, confidence, drivers, status = train_and_predict_economy(master_df)
            if not prediction: display(HTML(f"<div class='empty-state'>Could not generate a forecast.<br><i>Reason: {status}</i></div>")); return
            reco = STRATEGIC_RECOMMENDATIONS.get(prediction)
            drivers_html = "".join([f"<span class='advisor-driver-tag' style='background-color:{CATEGORY_COLORS.get(d)}; color: #111827;'>{d}</span>" for d in drivers])
            biz_reco_html = "".join([f"<li>{item}</li>" for item in reco['biz_advice']])
            policy_reco_html = "".join([f"<li>{item}</li>" for item in reco['policy_advice']])
            advisor_html = f"""<div class="advisor-container"><div class="advisor-header"><h2 style="font-size: 24px; margin-bottom: 4px;">AI Economic Advisor: Near-Term Outlook</h2><p style="color: var(--text-alt);">Forecast based on analysis of recent global news events and market sentiment.</p></div><div class="advisor-prediction-card" style="border-left-color: {reco['color']};"><h3 class="advisor-prediction-title">{reco['icon']} {prediction}</h3><p class="advisor-prediction-desc">{reco['desc']}</p></div><div class="advisor-details"><div class="advisor-detail-card"><p class="advisor-detail-title">MODEL CONFIDENCE</p><p class="advisor-confidence" style="color: {reco['color']};">{confidence:.1%}</p></div><div class="advisor-detail-card"><p class="advisor-detail-title">KEY RISK DRIVERS</p><div class="advisor-drivers">{drivers_html}</div></div></div><div class="advisor-reco-grid"><div class="reco-card"><h3>Strategic Recommendations for Businesses</h3><ul>{biz_reco_html}</ul></div><div class="reco-card"><h3>Strategic Recommendations for Policymakers</h3><ul>{policy_reco_html}</ul></div></div><hr style="border-color: var(--border); margin: 32px 0;"><div class="methodology-section"><h3 class="methodology-title">AI Methodology Breakdown (The "Explanation Facility")</h3><div class="methodology-grid"><div class="methodology-card"><h4 class="methodology-card-title">1. Knowledge Representation & NLP</h4><p class="methodology-card-desc">The system ingests and performs semantic analysis on thousands of news articles to understand their content.</p><p class="methodology-card-syllabus">Syllabus Topics: #8 (NLP), #6 (Knowledge Acquisition)</p></div><div class="methodology-card"><h4 class="methodology-card-title">2. Rule-Based Reasoning</h4><p class="methodology-card-desc">An expert system uses a knowledge base of keywords to classify each article and assign a risk score, creating structured data from text.</p><p class="methodology-card-syllabus">Syllabus Topics: #5 (Rule-Based Systems), #4 (First-Order Logic)</p></div><div class="methodology-card"><h4 class="methodology-card-title">3. Inductive Learning</h4><p class="methodology-card-desc">A model composed of multiple Decision Trees learns the patterns between daily risk scores and real-world economic fear (the VIX index).</p><p class="methodology-card-syllabus">Syllabus Topics: #7 (Learning Decision Trees)</p></div><div class="methodology-card"><h4 class="methodology-card-title">4. Probabilistic Forecasting</h4><p class="methodology-card-desc">The trained model analyzes the most recent risk data to predict the future economic outlook with a calculated probability (confidence score).</p><p class="methodology-card-syllabus">Syllabus Topics: #5 (Probabilistic Reasoning)</p></div></div></div></div>"""
            display(HTML(advisor_html))

    def on_pipeline_run_click(b):
        nonlocal master_df
        b.disabled = True; b.icon = 'spinner'
        master_df = run_automated_pipeline()
        render_news_feed(); render_event_tracker(None); render_map(); render_trends(); render_market_pulse(); render_advisor_tab()
        b.disabled = False; b.icon = 'cloud-download'
    def on_category_filter_click(b): nonlocal active_category_filter; active_category_filter = b.description; [btn.remove_class('active') for btn in category_filter_buttons.values()]; b.add_class('active'); render_news_feed()
    def on_source_filter_click(b): nonlocal active_source_filter; active_source_filter = b.description; [btn.remove_class('active') for btn in source_filter_buttons.values()]; b.add_class('active'); render_news_feed()
    def on_search_change(change): render_news_feed()
    def on_run_tracker_click(b): nonlocal master_df; path = find_event_path('Stable', goal_dropdown.value, master_df); render_event_tracker(path)

    search_box = widgets.Text(placeholder='Search news feed...', layout={'flex': '1 1 auto'}); search_box.observe(on_search_change, 'value')
    pipeline_button = Button(description='Run Pipeline', icon='cloud-download', button_style='primary', layout={'width':'180px'}); pipeline_button.on_click(on_pipeline_run_click)
    category_filter_buttons = {name: Button(description=name) for name in ['All'] + TARGET_CATEGORIES}; source_filter_buttons = {name: Button(description=name) for name in ['Global', 'Bangladeshi']}
    [b.add_class('filter-button') for b in category_filter_buttons.values()]; [b.add_class('filter-button') for b in source_filter_buttons.values()]
    category_filter_buttons['All'].add_class('active'); source_filter_buttons['Global'].add_class('active')
    [b.on_click(on_category_filter_click) for b in category_filter_buttons.values()]; [b.on_click(on_source_filter_click) for b in source_filter_buttons.values()]
    news_controls = VBox([HBox([search_box, pipeline_button]), HBox([HTML("<b class='filter-label'>Source:</b>")] + list(source_filter_buttons.values())), HBox([HTML("<b class='filter-label'>Category:</b>")] + list(category_filter_buttons.values()))])
    goal_dropdown = Dropdown(options=list(GEOPOLITICAL_STATES.keys()), value='Active Conflict', description='Simulate path to:', style={'description_width': 'initial'})
    run_tracker_button = Button(description="Run Simulation", icon='cogs', button_style='info'); run_tracker_button.on_click(on_run_tracker_click)
    tracker_controls = VBox([HBox([goal_dropdown, run_tracker_button], layout=Layout(align_items='center', padding="16px"))])

    tab = widgets.Tab(children=[news_output, tracker_output, map_output, trends_output, market_output, advisor_output])
    tab.set_title(0, '📰 Feed'); tab.set_title(1, '🛰️ Tracker'); tab.set_title(2, '🌍 Map'); tab.set_title(3, '📈 Trends'); tab.set_title(4, '💰 Markets'); tab.set_title(5, '🧠 AI Advisor')
    header_box = HTML("""<div class="dashboard-header"><div class="header-center"><h1 class="dashboard-title"><span>🌐</span> WorldWatch AI</h1></div><button class="theme-toggle" onclick="document.body.classList.toggle('light-mode'); this.textContent = this.textContent === '☀️' ? '🌙' : '☀️';">☀️</button></div>""")
    subtitle_box = HTML("""<div class="dashboard-header" style="padding-top:0; justify-content:center;"><p class="dashboard-subtitle">Your Global Intelligence Briefing</p></div>""")
    dashboard_container = VBox([css_html, header_box, subtitle_box, tab]); dashboard_container.add_class("worldwatch-dashboard")

    # --- Initial Render ---
    display(dashboard_container)
    render_news_feed(); render_map(); render_trends(); render_market_pulse(); render_advisor_tab()
    with tracker_output: display(tracker_controls); display(HTML("<div class='empty-state'>Run the pipeline first, then select a goal and run the simulation.</div>"))

# --- Run the application ---
create_dashboard()